In [1]:
import os
import SimpleITK as sitk
import math
import glob
import pandas as pd
import random
import time
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import PIL
import pydicom

In [2]:
images=os.listdir('test patient2')
images
for i in images:
    ct_path='{}/CT.nrrd'.format(i)
    CT_path='test patient2/{}'.format(ct_path)
    print(CT_path)
    ct=sitk.ReadImage(CT_path)
    ct_array=sitk.GetArrayFromImage(ct)
    #保存为{}_ct.npy
    number=ct_path[0:8]
    savepath='test input2\CT\{}_ct.npy'.format(number)
    np.save(savepath,ct_array)
    print(ct_array.shape)


test patient2/RT231416/CT.nrrd
(141, 512, 512)
test patient2/RT231774/CT.nrrd
(145, 512, 512)
test patient2/RT232101/CT.nrrd
(164, 512, 512)


In [3]:
#dose重采样到和CT一样大小
def align_coordinate(ori_img, target_img, resamplemethod=sitk.sitkNearestNeighbor):
    #     "  用itk方法将原始图像resample到与目标图像一致
    #    :param ori_img: 原始需要对齐的itk图像
    #    :param target_img: 要对齐的目标itk图像
    #    :param resamplemethod: itk插值方法: sitk.sitkLinear-线性  sitk.sitkNearestNeighbor-最近邻
    #    :return:img_res_itk: 重采样好的itk图像

    target_Size = target_img.GetSize()  # 目标图像大小  [x,y,z]
    target_Spacing = target_img.GetSpacing()  # 目标的体素块尺寸    [x,y,z]
    target_origin = target_img.GetOrigin()  # 目标的起点 [x,y,z]
    target_direction = target_img.GetDirection()  # 目标的方向 [冠,矢,横]=[z,y,x]

    # itk的方法进行resample
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ori_img)  # 需要重新采样的目标图像
    # 设置目标图像的信息
    resampler.SetSize(target_Size)  # 目标图像大小
    resampler.SetOutputOrigin(target_origin)
    resampler.SetOutputDirection(target_direction)
    resampler.SetOutputSpacing(target_Spacing)
    # 根据需要重采样图像的情况设置不同的dype
    if resamplemethod == sitk.sitkNearestNeighbor:
        resampler.SetOutputPixelType(sitk.sitkUInt16)  # 近邻插值用于mask的，保存uint16
    else:
        resampler.SetOutputPixelType(sitk.sitkFloat32)  # 线性插值用于PET/CT/MRI之类的，保存float32
    resampler.SetTransform(sitk.Transform(3, sitk.sitkIdentity))
    resampler.SetInterpolator(resamplemethod)
    itk_img_resampled = resampler.Execute(ori_img)  # 得到重新采样后的图像
    return itk_img_resampled

In [4]:
images=os.listdir('test patient2')
for i in images:
    #抓取dose的路径
    dose_path='{}/RD'.format(i)
    RD_path='test patient2/{}'.format(dose_path)
    path=os.listdir(RD_path)
    for n in path:
        ori_path='{}/{}'.format(RD_path,n)
    print(ori_path)
    ori_img=sitk.ReadImage(ori_path)
    #抓取CT的路径
    ct_path='{}/CT.nrrd'.format(i)
    target_path='test patient2/{}'.format(ct_path)
    print(target_path)
    target_img=sitk.ReadImage(target_path)
    return_img=align_coordinate(ori_img, target_img, resamplemethod=sitk.sitkNearestNeighbor)

    return_array=sitk.GetArrayFromImage(return_img)
    i=dose_path[0:8]
    savepath='test output2\{}_rd.npy'.format(i)
    np.save(savepath,return_array)

test patient2/RT231416/RD/95487635_Cervixfinal_Dose.dcm
test patient2/RT231416/CT.nrrd
test patient2/RT231774/RD/95513677_CervixPlan_Dose.dcm
test patient2/RT231774/CT.nrrd
test patient2/RT232101/RD/1816939_Cervixfinal_Dose.dcm
test patient2/RT232101/CT.nrrd
